In [1]:
from extraction.tournament import Tournament
import pandas as pd
from mplsoccer import VerticalPitch, Pitch, Radar, FontManager, grid
import matplotlib.pyplot as plt
import numpy as np
from pandas import *
import matplotlib.cm as cm
from adjustText import adjust_text


In [2]:
# brasileirao_2024 = Tournament("Brasileirao", 325,58766)
# all_players_df_2 = brasileirao_2024.fetch_live_tournament_stats_from_all_players()

# import pandas as pd

# Carregar o arquivo .xlsx em um DataFrame
file_path = 'jogadores_do_brasileirao.xlsx'
all_players_df = pd.read_excel(file_path)


In [3]:
# # peguei os dados dia 07/10

# all_players_df.to_excel('jogadores_do_brasileirao.xlsx', index = False)
# all_players_df = all_players_df



In [4]:
all_players_df.columns.tolist()

['player_name',
 'team_name',
 'primary_color',
 'secondary_color',
 'badge_url',
 'avatar_url',
 'rating',
 'goals',
 'assists',
 'expectedGoals',
 'expectedAssists',
 'goalsAssistsSum',
 'accuratePasses',
 'inaccuratePasses',
 'totalPasses',
 'accuratePassesPercentage',
 'accurateFinalThirdPasses',
 'keyPasses',
 'touches',
 'successfulDribbles',
 'successfulDribblesPercentage',
 'interceptions',
 'yellowCards',
 'directRedCards',
 'redCards',
 'accurateCrosses',
 'accurateCrossesPercentage',
 'totalShots',
 'shotsOnTarget',
 'shotsOffTarget',
 'shotsFromOutsideTheBox',
 'goalsFromOutsideTheBox',
 'shotsFromInsideTheBox',
 'goalsFromInsideTheBox',
 'aerialDuelsWon',
 'aerialDuelsWonPercentage',
 'totalDuelsWon',
 'totalDuelsWonPercentage',
 'minutesPlayed',
 'goalConversionPercentage',
 'penaltiesTaken',
 'penaltyGoals',
 'shotFromSetPiece',
 'accurateLongBalls',
 'accurateLongBallsPercentage',
 'clearances',
 'errorLeadToShot',
 'wasFouled',
 'fouls',
 'dribbledPast',
 'offsides',
 

In [5]:
# all_players_df['nineties'] = all_players_df['minutesPlayed'] / 90


# # Colunas que serão ignoradas na criação de colunas Per90
# ignore_columns = [
#     'appearances', 'scoringFrequency', 'substitutionsIn', 'substitutionsOut', 
#     'penaltyConversion', 'matchesStarted', 'cleanSheet', 
#     'accurateLongBallsPercentage', 'goalConversionPercentage', 'minutesPlayed', 
#     'totalDuelsWonPercentage', 'aerialDuelsWonPercentage', 'accurateCrossesPercentage', 
#     'successfulDribblesPercentage', 'accuratePassesPercentage'
# ]

# # Filtrar colunas que não devem ser convertidas para Per90
# columns_to_convert = [
#     col for col in all_players_df.columns 
#     if col not in ignore_columns and col not in ['player_name', 'team_name', 'primary_color', 'secondary_color', 'badge_url', 'avatar_url', 'rating', 'nineties']
# ]

# # Criar novo DataFrame com as colunas Per90, evitando duplicação
# per90_columns = {
#     f'{col}Per90': all_players_df[col] / all_players_df['nineties'] 
#     for col in columns_to_convert
#     if f'{col}Per90' not in all_players_df.columns  # Garante que não crie colunas duplicadas
# }

# # Concatenar as novas colunas Per90 ao DataFrame original
# all_players_df = pd.concat([all_players_df, pd.DataFrame(per90_columns)], axis=1)

# # Reorganizar para desfazer fragmentação, se necessário
# all_players_df = all_players_df.copy()



# all_players_df

In [6]:
# # all_players_df['xgPerShot'] = all_players_df['expectedGoals'] / all_players_df['totalShots']

# all_players_df['touchesPer90'] = all_players_df['touches'] / all_players_df['nineties']

In [7]:
def scatter_plot_sofascore(stats_df, x_values, y_values, highlights = [], references = [], players = True, min_nineties = 3, remove_zeros = True, name_font_size = 8.5, show_title = True, hl_x_displacement = 0.0, hl_y_displacement = 0.12, vl_x_displacement = -1.0, vl_y_displacement = 0.0, name_x_displacement = 0.2, name_y_displacement = 0.2, scatter_multiplier = 1, start_x = 0, start_y = 0):

    column_names_dict = {
        'player_name': 'Player Name',
        'team_name': 'Team Name',
        'primary_color': 'Primary Color',
        'secondary_color': 'Secondary Color',
        'badge_url': 'Badge URL',
        'avatar_url': 'Avatar URL',
        'rating': 'Rating',
        'goals': 'Goals',
        'goalsPer90': 'Goals per 90',
        'assists': 'Assists',
        'assistsPer90': 'Assists per 90',
        'expectedGoals': 'Expected Goals',
        'expectedGoalsPer90': 'Expected Goals per 90',
        'xgPerShot': 'Expected Goals per Shot',
        'shotsFromOutsideTheBoxPer90':  'Shots from Outside The Box per 90',
        'goalsFromOutsideTheBoxPer90':  'Goals from Outside The Box per 90',
        'shotsFromInsideTheBoxPer90':  'Shots from Inside The Box per 90',
        'goalsFromInsideTheBoxPer90': 'Goals from Inside The Box per 90',
        'goalsAssistsSum': 'Goals + Assists',
        'goalsAssistsSumPer90': 'Goals + Assists per 90',
        'accuratePasses': 'Accurate Passes',
        'accuratePassesPer90': 'Accurate Passes per 90',
        'inaccuratePasses': 'Inaccurate Passes',
        'accuratePassesPercentage': 'Accurate Passes (%)',
        'inaccuratePassesPer90': 'Inaccurate Passes per 90',
        'totalPasses': 'Total Passes',
        'totalPassesPer90': 'Total Passes per 90',
        'accurateFinalThirdPasses': 'Accurate Final Third Passes',
        'accurateFinalThirdPassesPer90': 'Accurate Final Third Passes per 90',
        'keyPasses': 'Key Passes',
        'keyPassesPer90': 'Key Passes per 90',
        'successfulDribbles': 'Successful Dribbles',
        'successfulDribblesPer90': 'Successful Dribbles per 90',
        'successfulDribblesPercentage': 'Successful Dribbles (%)',
        'interceptions': 'Interceptions',
        'interceptionsPer90': 'Interceptions per 90',
        'yellowCards': 'Yellow Cards',
        'yellowCardsPer90': 'Yellow Cards per 90',
        'directRedCards': 'Direct Red Cards',
        'directRedCardsPer90': 'Direct Red Cards per 90',
        'redCards': 'Red Cards',
        'redCardsPer90': 'Red Cards per 90',
        'accurateCrosses': 'Accurate Crosses',
        'accurateCrossesPer90': 'Accurate Crosses per 90',
        'accurateCrossesPercentage': 'Cross Accuracy (%)',
        'totalShots': 'Total Shots',
        'totalShotsPer90': 'Total Shots per 90',
        'shotsOnTarget': 'Shots on Target',
        'shotsOnTargetPer90': 'Shots on Target per 90',
        'shotsOffTarget': 'Shots off Target',
        'shotsOffTargetPer90': 'Shots off Target per 90',
        'aerialDuelsWon': 'Aerial Duels Won',
        'aerialDuelsWonPer90': 'Aerial Duels Won per 90',
        'aerialDuelsWonPercentage': 'Aerial Duels Won (%)',
        'totalDuelsWon': 'Total Duels Won',
        'totalDuelsWonPer90': 'Total Duels Won per 90',
        'totalDuelsWonPercentage': 'Duels Won (%)',
        'clearances': 'Clearances',
        'clearancesPer90': 'Clearances per 90',
        'errorLeadToShot': 'Error Leading to Shot',
        'errorLeadToShotPer90': 'Error Leading to Shot per 90',
        'wasFouled': 'Was Fouled',
        'wasFouledPer90': 'Was Fouled per 90',
        'fouls': 'Fouls',
        'foulsPer90': 'Fouls per 90',
        'dribbledPast': 'Dribbled Past',
        'dribbledPastPer90': 'Dribbled Past per 90',
        'offsides': 'Offsides',
        'offsidesPer90': 'Offsides per 90',
        'blockedShots': 'Blocked Shots',
        'blockedShotsPer90': 'Blocked Shots per 90',
        'passToAssist': 'Pass to Assist',
        'passToAssistPer90': 'Pass to Assist per 90',
        'saves': 'Saves',
        'savesPer90': 'Saves per 90',
        'cleanSheet': 'Clean Sheets',
        'totalCross': 'Total Crosses',
        'totalCrossPer90': 'Total Crosses per 90',
        'duelLost': 'Duels Lost',
        'duelLostPer90': 'Duels Lost per 90',
        'aerialLost': 'Aerial Duels Lost',
        'aerialLostPer90': 'Aerial Duels Lost per 90',
        'attemptPenaltyMiss': 'Penalty Attempts Missed',
        'attemptPenaltyMissPer90': 'Penalty Attempts Missed per 90',
        'totalLongBalls': 'Total Long Balls',
        'totalLongBallsPer90': 'Total Long Balls per 90',
        'accurateLongBallsPercentage': 'Long Ball Accuracy (%)',
        'goalsConceded': 'Goals Conceded',
        'goalsConcededPer90': 'Goals Conceded per 90',
        'goalConversionPercentage': 'Goal Conversion (%)',
        'goalKicks': 'Goal Kicks',
        'goalKicksPer90': 'Goal Kicks per 90',
        'ballRecovery': 'Ball Recoveries',
        'ballRecoveryPer90': 'Ball Recoveries per 90',
        'appearances': 'Appearances',
        'scoringFrequency': 'Scoring Frequency',
        'yellowRedCards': 'Yellow + Red Cards',
        'substitutionsIn': 'Substitutions In',
        'substitutionsOut': 'Substitutions Out',
        'matchesStarted': 'Matches Started',
        'penaltyConversion': 'Penalty Conversion (%)',
        'minutesPlayed': 'Minutes Played'
}



    URL6 = 'https://github.com/adobe/ferrumjsorg/blob/main/fonts/Montserrat-Regular.ttf?raw=true'
    montserrat = FontManager(URL6)

    URL7 = 'https://github.com/JulietaUla/Montserrat/blob/master/fonts/otf/Montserrat-Bold.otf?raw=true'
    montserrat_bold = FontManager(URL7)

    df = stats_df

    # Minutagem mínima e tirando jogadores que tenham 0 nas estatísticas a se comparar (caso necessário):
    if (players and min_nineties):
        df = df[df['nineties'] >= min_nineties].reset_index(drop = True)

    if remove_zeros:
        df = df[df[x_values] != 0].reset_index(drop = True)
        df = df[df[y_values] != 0].reset_index(drop = True)


    # Criando o scatter plot
    fig, ax = plt.subplots(figsize=(12, 8), dpi=1000, facecolor='#1E2836')

    # Mudar a cor do fundo da área do gráfico
    ax.set_facecolor('#1E2836')

    # Plotando os pontos:

    plt.scatter(df[x_values], df[y_values], color='red', edgecolors='darkred', linewidths=0.25, s=12*scatter_multiplier)

    ax.set_xlim(left=start_x)
    ax.set_ylim(bottom=start_y)

    # Adicionando as linhas de média
    plt.axhline(df[y_values].mean(), color='gray', linestyle='dotted')
    plt.text(hl_x_displacement + start_x, df[y_values].mean() + hl_y_displacement, ' Average', fontsize=8, fontproperties=montserrat_bold.prop, color = 'darkgray')

    plt.axvline(df[x_values].mean(), color='gray', linestyle='dotted', label='Média X')
    plt.text(df[x_values].mean() + vl_x_displacement, vl_y_displacement + start_y, '  Average', fontsize=8, fontproperties=montserrat_bold.prop, color = 'darkgray', rotation = 90)

    # Anotando os nomes e mudando a cor das jogadoras/times em destaque

    for i in range(len(df)):

        if players:

            if df['player_name'][i] in highlights:
                plt.text(df[x_values][i] + name_x_displacement, df[y_values][i] + name_y_displacement, df['player_name'][i], fontsize=name_font_size, fontproperties=montserrat_bold.prop, color = 'white')
                plt.scatter(df[x_values][i], df[y_values][i], color='cyan', edgecolors='darkturquoise', linewidths=0.25, s=18*scatter_multiplier)

            if df['player_name'][i] in references:
                plt.text(df[x_values][i] + name_x_displacement, df[y_values][i] + name_y_displacement, df['player_name'][i], fontsize=name_font_size, fontproperties=montserrat_bold.prop, color = 'gainsboro')
                plt.scatter(df[x_values][i], df[y_values][i], color='seagreen', edgecolors='green', linewidths=0.25, s=18*scatter_multiplier)

        else:
            if df['team_name'][i] in highlights:
                plt.text(df[x_values][i] + name_x_displacement, df[y_values][i] + name_y_displacement, df['team_name'][i], fontsize=name_font_size, fontproperties=montserrat_bold.prop, color = 'white')
                plt.scatter(df[x_values][i], df[y_values][i], color='cyan', edgecolors='darkturquoise', linewidths=0.25, s=18*scatter_multiplier)


    # Adicionando títulos e rótulos
    if show_title:
        plt.title(f'{column_names_dict[x_values]} x {column_names_dict[y_values]}', loc = 'left', fontproperties=montserrat_bold.prop, color = 'white', fontsize = 18)
    plt.xlabel(column_names_dict[x_values], fontproperties=montserrat.prop, color = 'white')
    plt.ylabel(column_names_dict[y_values], fontproperties=montserrat.prop, color = 'white')

    plt.grid(visible=None, which='major', axis='both', color = 'darkgrey', linewidth = 0.08)

    # AJUSTES DE CORES:

    # Alterando as cores das linhas dos eixos horizontal e vertical

    ax.spines['top'].set_color('white')
    ax.spines['bottom'].set_color('white')
    ax.spines['left'].set_color('white')
    ax.spines['right'].set_color('white')

    # Alterando a cor dos ticks (marcadores dos eixos)
    ax.tick_params(colors='white', which='both')

    # Ajustando a fonte e cor dos números ao lado dos ticks

    plt.xticks(fontproperties=montserrat.prop, color = 'white')
    plt.yticks(fontproperties=montserrat.prop, color = 'white')

    # Mostrando o gráfico
    plt.show()

In [39]:
from adjustText import adjust_text

def scatter_plot_sofascore(stats_df, x_values, y_values, highlights = [], references = [], players = True, min_nineties = 3, remove_zeros = True, name_font_size = 8.5, show_title = True, hl_x_displacement = 0.0, hl_y_displacement = 0.12, vl_x_displacement = -1.0, vl_y_displacement = 0.0, name_x_displacement = 0.2, name_y_displacement = 0.2, scatter_multiplier = 1, start_x = 0, start_y = 0, linear = False):

    column_names_dict = {
        'player_name': 'Player Name',
        'team_name': 'Team Name',
        'primary_color': 'Primary Color',
        'secondary_color': 'Secondary Color',
        'badge_url': 'Badge URL',
        'avatar_url': 'Avatar URL',
        'rating': 'Rating',
        'goals': 'Goals',
        'goalsPer90': 'Goals per 90',
        'assists': 'Assists',
        'assistsPer90': 'Assists per 90',
        'expectedGoals': 'Expected Goals',
        'expectedGoalsPer90': 'Expected Goals per 90',
        'expectedAssistsPer90': 'Expected Assists per 90',
        'xgPerShot': 'Expected Goals per Shot',
        'shotsFromOutsideTheBoxPer90':  'Shots from Outside The Box per 90',
        'goalsFromOutsideTheBoxPer90':  'Goals from Outside The Box per 90',
        'shotsFromInsideTheBoxPer90':  'Shots from Inside The Box per 90',
        'goalsFromInsideTheBoxPer90': 'Goals from Inside The Box per 90',
        'goalsAssistsSum': 'Goals + Assists',
        'goalsAssistsSumPer90': 'Goals + Assists per 90',
        'accuratePasses': 'Accurate Passes',
        'accuratePassesPer90': 'Accurate Passes per 90',
        'inaccuratePasses': 'Inaccurate Passes',
        'accuratePassesPercentage': 'Accurate Passes (%)',
        'inaccuratePassesPer90': 'Inaccurate Passes per 90',
        'totalPasses': 'Total Passes',
        'totalPassesPer90': 'Total Passes per 90',
        'accurateFinalThirdPasses': 'Accurate Final Third Passes',
        'accurateFinalThirdPassesPer90': 'Accurate Final Third Passes per 90',
        'keyPasses': 'Key Passes',
        'keyPassesPer90': 'Key Passes per 90',
        'successfulDribbles': 'Successful Dribbles',
        'successfulDribblesPer90': 'Successful Dribbles per 90',
        'successfulDribblesPercentage': 'Successful Dribbles (%)',
        'interceptions': 'Interceptions',
        'interceptionsPer90': 'Interceptions per 90',
        'yellowCards': 'Yellow Cards',
        'yellowCardsPer90': 'Yellow Cards per 90',
        'directRedCards': 'Direct Red Cards',
        'directRedCardsPer90': 'Direct Red Cards per 90',
        'redCards': 'Red Cards',
        'redCardsPer90': 'Red Cards per 90',
        'accurateCrosses': 'Accurate Crosses',
        'accurateCrossesPer90': 'Accurate Crosses per 90',
        'accurateCrossesPercentage': 'Cross Accuracy (%)',
        'totalShots': 'Total Shots',
        'totalShotsPer90': 'Total Shots per 90',
        'shotsOnTarget': 'Shots on Target',
        'shotsOnTargetPer90': 'Shots on Target per 90',
        'shotsOffTarget': 'Shots off Target',
        'shotsOffTargetPer90': 'Shots off Target per 90',
        'aerialDuelsWon': 'Aerial Duels Won',
        'aerialDuelsWonPer90': 'Aerial Duels Won per 90',
        'aerialDuelsWonPercentage': 'Aerial Duels Won (%)',
        'totalDuelsWon': 'Total Duels Won',
        'totalDuelsWonPer90': 'Total Duels Won per 90',
        'totalDuelsWonPercentage': 'Duels Won (%)',
        'clearances': 'Clearances',
        'clearancesPer90': 'Clearances per 90',
        'errorLeadToShot': 'Error Leading to Shot',
        'errorLeadToShotPer90': 'Error Leading to Shot per 90',
        'wasFouled': 'Was Fouled',
        'wasFouledPer90': 'Was Fouled per 90',
        'fouls': 'Fouls',
        'foulsPer90': 'Fouls per 90',
        'dribbledPast': 'Dribbled Past',
        'dribbledPastPer90': 'Dribbled Past per 90',
        'offsides': 'Offsides',
        'offsidesPer90': 'Offsides per 90',
        'blockedShots': 'Blocked Shots',
        'blockedShotsPer90': 'Blocked Shots per 90',
        'passToAssist': 'Pass to Assist',
        'passToAssistPer90': 'Pass to Assist per 90',
        'saves': 'Saves',
        'savesPer90': 'Saves per 90',
        'cleanSheet': 'Clean Sheets',
        'totalCross': 'Total Crosses',
        'totalCrossPer90': 'Total Crosses per 90',
        'duelLost': 'Duels Lost',
        'duelLostPer90': 'Duels Lost per 90',
        'aerialLost': 'Aerial Duels Lost',
        'aerialLostPer90': 'Aerial Duels Lost per 90',
        'attemptPenaltyMiss': 'Penalty Attempts Missed',
        'attemptPenaltyMissPer90': 'Penalty Attempts Missed per 90',
        'totalLongBalls': 'Total Long Balls',
        'totalLongBallsPer90': 'Total Long Balls per 90',
        'accurateLongBallsPercentage': 'Long Ball Accuracy (%)',
        'goalsConceded': 'Goals Conceded',
        'goalsConcededPer90': 'Goals Conceded per 90',
        'goalConversionPercentage': 'Goal Conversion (%)',
        'goalKicks': 'Goal Kicks',
        'goalKicksPer90': 'Goal Kicks per 90',
        'ballRecovery': 'Ball Recoveries',
        'ballRecoveryPer90': 'Ball Recoveries per 90',
        'appearances': 'Appearances',
        'scoringFrequency': 'Scoring Frequency',
        'yellowRedCards': 'Yellow + Red Cards',
        'substitutionsIn': 'Substitutions In',
        'substitutionsOut': 'Substitutions Out',
        'matchesStarted': 'Matches Started',
        'penaltyConversion': 'Penalty Conversion (%)',
        'minutesPlayed': 'Minutes Played',
        'touchesPer90': 'Touches Per 90'
}


    URL6 = 'https://github.com/adobe/ferrumjsorg/blob/main/fonts/Montserrat-Regular.ttf?raw=true'
    montserrat = FontManager(URL6)

    URL7 = 'https://github.com/JulietaUla/Montserrat/blob/master/fonts/otf/Montserrat-Bold.otf?raw=true'
    montserrat_bold = FontManager(URL7)

    df = stats_df

    # Minutagem mínima e tirando jogadores que tenham 0 nas estatísticas a se comparar (caso necessário):
    if (players and min_nineties):
        df = df[df['nineties'] >= min_nineties].reset_index(drop = True)

    if remove_zeros:
        df = df[df[x_values] != 0].reset_index(drop = True)
        df = df[df[y_values] != 0].reset_index(drop = True)

    # Criando o scatter plot
    fig, ax = plt.subplots(figsize=(12, 8), dpi=1000, facecolor='#1E2836')
    ax.set_facecolor('#1E2836')

    # Plotando os pontos:
    plt.scatter(df[x_values], df[y_values], color='red', edgecolors='darkred', linewidths=0.25, s=12*scatter_multiplier)

    ax.set_xlim(left=start_x)
    ax.set_ylim(bottom=start_y)

    # Adicionando linha média ou linha y = x
    if linear:
        # Adicionando a linha y = x
        max_value = max(df[x_values].max(), df[y_values].max())  # Pegando o maior valor entre as duas colunas para ajustar os limites
        plt.plot([start_x, max_value], [start_y, max_value], color='gray', linestyle='dotted', label='y = x')
        #plt.text(max_value - 1, max_value - 1, ' y = x', fontsize=8, fontproperties=montserrat_bold.prop, color='darkgray')
    else:
        # Adicionando as linhas de média
        plt.axhline(df[y_values].mean(), color='gray', linestyle='dotted')
        plt.text(hl_x_displacement + start_x, df[y_values].mean() + hl_y_displacement, ' Average', fontsize=8, fontproperties=montserrat_bold.prop, color='darkgray')

        plt.axvline(df[x_values].mean(), color='gray', linestyle='dotted', label='Média X')
        plt.text(df[x_values].mean() + vl_x_displacement, vl_y_displacement + start_y, '  Average', fontsize=8, fontproperties=montserrat_bold.prop, color='darkgray', rotation=90)

    # # Adicionando as linhas de média
    # plt.axhline(df[y_values].mean(), color='gray', linestyle='dotted')
    # plt.text(hl_x_displacement + start_x, df[y_values].mean() + hl_y_displacement, ' Average', fontsize=8, fontproperties=montserrat_bold.prop, color = 'darkgray')

    # plt.axvline(df[x_values].mean(), color='gray', linestyle='dotted', label='Média X')
    # plt.text(df[x_values].mean() + vl_x_displacement, vl_y_displacement + start_y, '  Average', fontsize=8, fontproperties=montserrat_bold.prop, color = 'darkgray', rotation = 90)

    # Armazenando as anotações para ajuste
    texts = []

    # Anotando os nomes e mudando a cor das jogadoras/times em destaque
    for i in range(len(df)):
        if players:
            if df['player_name'][i] in highlights:
                texts.append(plt.text(df[x_values][i] + name_x_displacement, df[y_values][i] + name_y_displacement, df['player_name'][i], fontsize=name_font_size, fontproperties=montserrat_bold.prop, color = 'white'))
                plt.scatter(df[x_values][i], df[y_values][i], color='cyan', edgecolors='darkturquoise', linewidths=0.25, s=24*scatter_multiplier)
            elif df['player_name'][i] in references:
                texts.append(plt.text(df[x_values][i] + name_x_displacement, df[y_values][i] + name_y_displacement, df['player_name'][i], fontsize=name_font_size, fontproperties=montserrat_bold.prop, color = 'gainsboro'))
                plt.scatter(df[x_values][i], df[y_values][i], color='seagreen', edgecolors='green', linewidths=0.25, s=16*scatter_multiplier)
        else:
            if df['team_name'][i] in highlights:
                texts.append(plt.text(df[x_values][i] + name_x_displacement, df[y_values][i] + name_y_displacement, df['team_name'][i], fontsize=name_font_size, fontproperties=montserrat_bold.prop, color = 'white'))
                plt.scatter(df[x_values][i], df[y_values][i], color='cyan', edgecolors='darkturquoise', linewidths=0.25, s=18*scatter_multiplier)

    # Ajustando a posição dos textos para evitar sobreposição
    adjust_text(texts, arrowprops=dict(arrowstyle="-", color='lightgray', lw=0.5))

    # Adicionando títulos e rótulos
    if show_title:
        plt.title(f'{column_names_dict[x_values]} x {column_names_dict[y_values]}', loc = 'left', fontproperties=montserrat_bold.prop, color = 'white', fontsize = 18)
    plt.xlabel(column_names_dict[x_values], fontproperties=montserrat.prop, color = 'white')
    plt.ylabel(column_names_dict[y_values], fontproperties=montserrat.prop, color = 'white')

    plt.grid(visible=None, which='major', axis='both', color = 'darkgrey', linewidth = 0.015)

    # Ajustes de cores
    ax.spines['top'].set_color('white')
    ax.spines['bottom'].set_color('white')
    ax.spines['left'].set_color('white')
    ax.spines['right'].set_color('white')
    ax.tick_params(colors='white', which='both')

    plt.xticks(fontproperties=montserrat.prop, color = 'white')
    plt.yticks(fontproperties=montserrat.prop, color = 'white')

    # Mostrando o gráfico
    plt.show()


In [9]:
# # Verifica se existe um jogador chamado 'Wesley' no time 'Corinthians'
# condition = (all_players_df['player_name'] == 'Wesley') & (all_players_df['team_name'] == 'corinthians')

# # Renomeia o 'Wesley' do time 'Corinthians' para 'Wesley Gassova'
# all_players_df.loc[condition, 'player_name'] = 'Wesley Gassova'

# # Verifica se a alteração foi realizada
# print(all_players_df[all_players_df['player_name'].str.contains('Wesley')])


In [10]:
all_players_df.to_excel('jogadores_do_brasileirao.xlsx', index = False)
all_players_df = all_players_df


In [17]:
atacantes_destaque = ['Luiz Henrique', 'Felipe Anderson', 'Gustavo Scarpa', 'Breno Lopes', 'André Silva', 'Everton Ribeiro',
                    'Gabriel Veron', 'David', 'Yefferson Soteldo', 'Wesley', 'Rodrigo Garro', 'Yannick Bolasie', 'Lucas Barbosa', 'Gonzalo Mastriani']

atacantes_referencia = ['Pedro', 'Giorgian de Arrascaeta', 'Pablo Vegetti', 'Hulk', 'Paulinho', 'José López', 'Estêvão', 'Raphael Veiga', 'Matheus Pereira',
                        'Yuri Alberto', 'Lucas Moura', 'Ángel Romero']


meias_destaque = ['Gregore', 'Nicolás de la Cruz', 'Aníbal Moreno', 'Fausto Vera', 'Matheus Rossetto', 'Luiz Gustavo', 'Jean Lucas', 'Lucas Romero',
                   'Hugo Moura', 'Dodi', 'Thiago Maia', 'Marquinhos', 'Raniele', 'Matheusinho', 'Bruno Zapelli']

meias_referencia = ['André', 'Ganso', 'Marlon Freitas', 'Franco Cristaldo', 'Fernandinho', 'Gerson', 'Matheus Pereira', 'Nenê', 'Alisson', 'Erick Pulgar']

defensores_destaque = ['Alexander Barboza', 'Léo Ortiz', 'Júnior Alonso', 'Felipe Jonatan', 'Nahuel Ferraresi', 'Santiago Arias', 'Zé Ivaldo', 
                       'Paulo Henrique', 'Alexandro Bernabei', 'Thiago Silva', 'Cacá', 'Leonardo Godoy']

defensores_referencia = ['William', 'Fabrício Bruno', 'Léo Pereira', 'Wesley França', 'Rodrigo Battaglia', 'Bastos', 'Marçal']


In [40]:
scatter_plot_sofascore(all_players_df, x_values = 'expectedGoalsPer90', y_values = 'goalsPer90', highlights = atacantes_destaque, references=atacantes_referencia, min_nineties=8, name_x_displacement=0, vl_x_displacement=-0.06, hl_y_displacement= 0.005,name_y_displacement=0, linear=True)

In [13]:

scatter_plot_sofascore(all_players_df, x_values = 'accurateFinalThirdPassesPer90', y_values = 'accuratePassesPercentage', highlights = atacantes_destaque, min_nineties=8, start_y=50, name_x_displacement=0.025, vl_x_displacement=-0.3)

In [14]:
all_players_df.columns.tolist()

['player_name',
 'team_name',
 'primary_color',
 'secondary_color',
 'badge_url',
 'avatar_url',
 'rating',
 'goals',
 'assists',
 'expectedGoals',
 'expectedAssists',
 'goalsAssistsSum',
 'accuratePasses',
 'inaccuratePasses',
 'totalPasses',
 'accuratePassesPercentage',
 'accurateFinalThirdPasses',
 'keyPasses',
 'touches',
 'successfulDribbles',
 'successfulDribblesPercentage',
 'interceptions',
 'yellowCards',
 'directRedCards',
 'redCards',
 'accurateCrosses',
 'accurateCrossesPercentage',
 'totalShots',
 'shotsOnTarget',
 'shotsOffTarget',
 'shotsFromOutsideTheBox',
 'goalsFromOutsideTheBox',
 'shotsFromInsideTheBox',
 'goalsFromInsideTheBox',
 'aerialDuelsWon',
 'aerialDuelsWonPercentage',
 'totalDuelsWon',
 'totalDuelsWonPercentage',
 'minutesPlayed',
 'goalConversionPercentage',
 'penaltiesTaken',
 'penaltyGoals',
 'shotFromSetPiece',
 'accurateLongBalls',
 'accurateLongBallsPercentage',
 'clearances',
 'errorLeadToShot',
 'wasFouled',
 'fouls',
 'dribbledPast',
 'offsides',
 